# Stock Sentiment Analysis

It will predict wheter stock will up or down

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/stocknews/Combined_News_DJIA.csv', encoding = 'ISO-8859-1')

# encoding is used to encode some of the utf-8 characters...so use it

In [ ]:
df.head()


In [ ]:
df.shape

### What the DATA is?
0 -> stock prize will have not going to increase nxt day

1 -> stok prize will increse on next day

TOP (1-25) --> headlines of new paper

In [ ]:
#divide in train and tes
train = df[df['Date']<'20150101']
test = df[df['Date'] > '20141231']

In [ ]:
train.shape

In [ ]:
test.shape

## Text Preprocessing

There are lots of 'symbols' so we will remove them from headline

In [ ]:
#remove punctuation
#apart from alphabets we will replace every character with blank
data = train.iloc[:, 2:27]
data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)

In [ ]:
#renaming column
list1 = [i for i in range(25)]
new_Index = [str(i) for i in list1]
data.columns = new_Index
data.head(5)

In [ ]:
#converting hadlines to lower case
for index in new_Index:
    data[index] = data[index].str.lower()
data.head(2)

> Combine all the 25 headlines to a single sentence

In [ ]:
' '.join(str(x) for x in data.iloc[1, 0:25])  #example

In [ ]:
#all the sentences in the form of list

headlines = []
for row in range(0, len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row, 0:25]))

In [ ]:
headlines[5]

Countvectorizer is used to convert all the sentences into vectors

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# implememt BAG OF WORDS
countVector = CountVectorizer(ngram_range=(2,2))
#ngram(2,2) means it will combine the 2 words together and assign the value

trainDataset = countVector.fit_transform(headlines)

In [ ]:
trainDataset[0]  #sparse matrix ie combination of 0 and 1

### Implement Random forest Classifier

In [ ]:
randomClassifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
randomClassifier.fit(trainDataset, train['Label'])


### Predict for the Test Dataset

In [ ]:
testTransform =[]
for row in range(0, len(test.index)):
    testTransform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))
    
test_dataset = countVector.transform(testTransform)
predictions = randomClassifier.predict(test_dataset)

In [ ]:
test.loc[1611, :]

In [ ]:
predictions

### Check accuracy


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)
score = accuracy_score(test['Label'], predictions)
print(score)
report = classification_report(test['Label'], predictions)
print(report)

# Lets use TFIDF instead of COunt Vectorizer


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# implememt BAG OF WORDS
tfidfVector = TfidfVectorizer(ngram_range=(2,2))
#ngram(2,2) means it will combine the 2 words together and assign the value

trainDataset = tfidfVector.fit_transform(headlines)

In [ ]:
randomClassifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
randomClassifier.fit(trainDataset, train['Label'])

In [ ]:
testTransform =[]
for row in range(0, len(test.index)):
    testTransform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))
    
test_dataset = tfidfVector.transform(testTransform)
predictions = randomClassifier.predict(test_dataset)

In [ ]:
predictions

In [ ]:
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)
score = accuracy_score(test['Label'], predictions)
print(score)
report = classification_report(test['Label'], predictions)
print(report)

# Lets use Navie bayes instead of RandomForest

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
naive = MultinomialNB()
naive.fit(trainDataset, train['Label'])

In [ ]:
predictions = naive.predict(test_dataset)
predictions

In [ ]:
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)
score = accuracy_score(test['Label'], predictions)
print(score)
report = classification_report(test['Label'], predictions)
print(report)